### Full POP model works

In [1]:
include("LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using DynamicPolynomials
using LinearAlgebra
using Dates
using HDF5

Consider simple Lindblad master equation with just one dissipator:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} e_1 & h_1 - i h_2 \\ h_1 + i h_2 & 0
   \end{pmatrix}
$

 we can possibly look for jump operator of the form:

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12} \\ a_{21} + i b_{21} & -a_{11} - i b_{11} 
   \end{pmatrix} 
$

or

$
A = \begin{pmatrix} a_{11} & a_{12} + i b_{12}  \\ a_{21} + i b_{21} & -a_{11}
   \end{pmatrix} 
$

or assuming zero temperature 

$
A = \begin{pmatrix} a_{11} + i b_{11} & a_{12}   \\ 0 & -a_{11} - i b_{11}
   \end{pmatrix} 
$

In [2]:
@polyvar e[1]
@polyvar h[1:2]


Hˢʸᵐᵇ = [ e[1]               h[1] - im*h[2]
          h[1] + im*h[2]     0.             ]

2×2 Matrix{Polynomial{true, ComplexF64}}:
 e₁                  h₁ + (0.0-1.0im)h₂
 h₁ + (0.0+1.0im)h₂  0.0+0.0im

In [9]:
method = "T0_diag0_fix-Phi-up_"

@polyvar a[1:4]
@polyvar b[1:4]


#"just-zero-down_ "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2] + im*b[2] 
#          0                -a[1]-im*b[1]  ]


#"fix-Phi-up_: "

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
#          a[3] + im*b[3]   -a[1]-im*b[1] ]


#"zero Temperature and fix-Phi-up_: " zero_down_fix-Phi-up_

#Aˢʸᵐᵇ = [ a[1]+im*b[1]      a[2]
#          0                -a[1]-im*b[1] ]

#"zero Temperature and fix-Phi-diag_: "

#Aˢʸᵐᵇ = [ a[1]     a[2]+im*b[2] 
#          0        -a[1]        ]

#"zero Temperature, 00-diagonal, fix-Phi-up_: "

Aˢʸᵐᵇ = [ 0       a[2] 
          0        0        ]




2×2 Matrix{Term{true, Int64}}:
 0  a₂
 0  0

In [4]:
pwd()

"/home/zah/POP23"

In [5]:
parentdir = pwd()
data_dir = parentdir*"/DATA/"
println(data_dir)

models_dir = parentdir*"/MODELS/"
tests_dir = parentdir*"/TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

/home/zah/POP23/DATA/


In [6]:
all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;

### Train models loop

In [7]:
import Base.real
function remove_micro_coefs(p::AbstractPolynomial, cutoff)
    sum(
        
        [abs(coef) > cutoff ? real(coef) * mon : 0 for (coef, mon) in zip(coefficients(p), monomials(p))]
        )
end

remove_micro_coefs (generic function with 1 method)

In [10]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
models_file = "POP_NoRmT0d0LME_trn4_gap_"*method*date_and_time_string * ".h5"
tests_data_file_name = "POP_NoRmT0d0LME_trn4_gap_tst20_"*method*date_and_time_string * ".h5"

println(models_file)

γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in γ
    
    println("---------------")
    println("gamma = ", γᵢ)
    
    Hᴸᴹᴱ = [ 25.126        0
             0             0    ]
    γᶠ = parse(Float64, γᵢ)

    Aᴸᴹᴱ = [ 0    √(γᶠ)
             0     0. + 0im  ]

    objₑₓ = 0
    
    objₗₘₑ = 0
    
    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        ρᵗʳⁿₒₚₛ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρᵗʳⁿ]
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])
    
        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
         
        
        ρˡᵐᵉ = LiPoSID.Lindblad_time_evolution(basis, ρᵗʳⁿ[1], tᵗʳⁿ, Hᴸᴹᴱ, [Aᴸᴹᴱ])
        objₗₘₑ += LiPoSID.simpson_obj(ρˡᵐᵉ, tᵗʳⁿ,  Hˢʸᵐᵇ, [Aˢʸᵐᵇ])
            
        
    end # of files (initial states) loop
    
    objₑₓrm = objₑₓ
    objₗₘₑrm = objₗₘₑ
    
    # if γᵢ == γ[1] #"0.079477" 
    #     objₑₓrm = remove_micro_coefs(objₑₓ, 5e-3)
    #     objₗₘₑrm = remove_micro_coefs(objₗₘₑ, 5e-5)
    # elseif  γᵢ == γ[2] #"0.25133" 
    #     objₑₓrm = remove_micro_coefs(objₑₓ, 1e-3)
    #     objₗₘₑrm = remove_micro_coefs(objₗₘₑ, 5e-7)           
    # elseif  γᵢ == γ[3] #"0.79477"
    #     objₑₓrm = remove_micro_coefs(objₑₓ, 5e-4)
    #     objₗₘₑrm = remove_micro_coefs(objₗₘₑ, 1e-6)
    # else 
    #     objₑₓrm = objₑₓ
    #     objₗₘₑrm = objₗₘₑ
    # end
            
    coef_minₑₓ = minimum(abs.(coefficients(objₑₓ)))
    coef_maxₑₓ = maximum(abs.(coefficients(objₑₓ)))
            
    cutoffₑₓ = log10(coef_maxₑₓ)-4

    coefs_gapₑₓ = coef_maxₑₓ/coef_minₑₓ 
            
    coef_minₗₘₑ = minimum(abs.(coefficients(objₗₘₑ)))
    coef_maxₗₘₑ = maximum(abs.(coefficients(objₗₘₑ)))
            
    cutoffₗₘₑ = log10(coef_maxₗₘₑ)-4

    coefs_gapₗₘₑ= coef_maxₗₘₑ/coef_minₗₘₑ 
       
    println("-------------------------------------------")
    println(" POP on ansatz LME simulation")
    println(cutoffₗₘₑ)
            
    #objₗₘₑrm = coefs_gapₗₘₑ > 1e6 ? remove_micro_coefs(objₗₘₑ, cutoffₗₘₑ) : objₗₘₑ
    
    solˢⁱᵈₗₘₑ, best_methodˢⁱᵈₗₘₑ = LiPoSID.sos_min_newton(objₗₘₑrm)

    
    println("Best metod for POP on LME: ", best_methodˢⁱᵈₗₘₑ)
    println("-------------------------------------------")
    
    solₗₘₑ = [e[1], h[1], h[2], a[1], a[2], a[3], b[1], b[3]] => [25.126, 0, 0, 0,  √(γᶠ), 0, 0, 0]
    
    obj_val_lme = convert(Float64, subs(objₗₘₑ, solₗₘₑ))
    
    obj_val_sid_on_lme = convert(Float64, subs(objₗₘₑ, solˢⁱᵈₗₘₑ))
    
    if obj_val_sid_on_lme < obj_val_lme
        println(" simulated LME overfitting")
    end
    
    Hˢⁱᵈₗₘₑ = subs(Hˢʸᵐᵇ, solˢⁱᵈₗₘₑ)
    Aˢⁱᵈₗₘₑ = subs(Aˢʸᵐᵇ, solˢⁱᵈₗₘₑ)
      
    println(" POP on ansatz EXACT Kurt data ")
    println(cutoffₑₓ)
                
    #objₑₓrm = coefs_gapₑₓ > 1e6 ? remove_micro_coefs(objₑₓ, cutoffₑₓ) : objₑₓ
    #objₑₓrm = coefs_gapₑₓ > 1e6 ? remove_micro_coefs(objₑₓ, -3) : objₑₓ
    
    solₑₓ, best_methodₑₓ = LiPoSID.sos_min_newton(objₑₓrm) 
    
    println("Best metod for POP on EXACT: ", best_methodₑₓ)
    
    obj_val_lme_on_exact = convert(Float64, subs(objₑₓ, solₗₘₑ))
    
    obj_val_sid_on_exact = convert(Float64, subs(objₑₓ, solₑₓ))
    
    if obj_val_sid_on_exact < obj_val_lme_on_exact
        println(" Exact data overfitting")
    end

    Hˢⁱᵈₑₓ = subs(Hˢʸᵐᵇ, solₑₓ)
    Aˢⁱᵈₑₓ = subs(Aˢʸᵐᵇ, solₑₓ)
    
    H = DenseOperator(basis, Hˢⁱᵈₑₓ)
    A = DenseOperator(basis, Aˢⁱᵈₑₓ)

    
    h5open(models_dir*models_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)
        
        
        γ_group["coef_min_ex"] = coef_minₑₓ
        γ_group["coef_max_ex"] = coef_maxₑₓ
        γ_group["coef_gap_ex"] = coefs_gapₑₓ
                        
        γ_group["coef_min_lme"] = coef_minₗₘₑ
        γ_group["coef_max_lme"] = coef_maxₗₘₑ
        γ_group["coef_gap_lme"] = coefs_gapₗₘₑ       
                        
        γ_group["obj_val_lme"] = obj_val_lme
        γ_group["obj_val_sid_on_lme"] = obj_val_sid_on_lme
        
        γ_group["obj_val_lme_on_exact"] = obj_val_lme_on_exact
        γ_group["obj_val_sid_on_exact"] = obj_val_sid_on_exact


        γ_group["H_sid_exact"] = convert.(ComplexF64, Hˢⁱᵈₑₓ)
        γ_group["A_sid_exact"] = convert.(ComplexF64, Aˢⁱᵈₑₓ)
        
        γ_group["H_lme"] = convert.(ComplexF64, Hᴸᴹᴱ)
        γ_group["A_lme"] = convert.(ComplexF64, Aᴸᴹᴱ)
        
        γ_group["H_sid_lme"] = convert.(ComplexF64, Hˢⁱᵈₗₘₑ)
        γ_group["A_sid_lme"] = convert.(ComplexF64, Aˢⁱᵈₗₘₑ)

        
    end # of HDF5 writing
    
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 
    
    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tᴸᴹᴱ, ρᴸᴹᴱ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hᴸᴹᴱ), [DenseOperator(basis,Aᴸᴹᴱ)])
        tˢⁱᵈₗₘ, ρˢⁱᵈₗₘₑ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑ), [DenseOperator(basis,Aˢⁱᵈₗₘₑ)])
        tˢⁱᵈₑₓ, ρˢⁱᵈₑₓ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₑₓ), [DenseOperator(basis,Aˢⁱᵈₑₓ)])   
        

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρᴸᴹᴱ)]
        Fˢⁱᵈᴸᴹᴱₗₘₑ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρˢⁱᵈₗₘₑ, ρᴸᴹᴱ)]
        Fˢⁱᵈᵉˣₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈₑₓ)]
                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_lme_exact"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["F_sidlme_lme"] = convert.(Float64, Fˢⁱᵈᴸᴹᴱₗₘₑ)
            init_state_group["F_sidexact_exact"] = convert.(Float64, Fˢⁱᵈᵉˣₑₓ)
            
        end
                    
        print(minimum(Fᴸᴹᴱₑₓ), minimum(Fˢⁱᵈᴸᴹᴱₗₘₑ), minimum(Fˢⁱᵈᵉˣₑₓ))
        println(" ")
    
    end
                
    println()
    println()
                            

end

POP_NoRmT0d0LME_trn4_gap_T0_diag0_fix-Phi-up_2023-Aug-10_at_17-26.h5
---------------
gamma = 0.079477
-------------------------------------------
 POP on ansatz LME simulation
-1.306163553400041
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = 1.4732054544616481e-8

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified with relative optimality gap 0.000001%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023


State_D3 0.99882914759186530.99999999999978580.9988545685842066 
State_D4 0.99883459456136880.99999999999978710.9988603617814619 
State_D5 0.99827581834072980.99999999999990830.9982948699353187 
State_D6 0.99827600330230990.99999999999990760.998295203760565 
State_D7 0.99850148794723810.999999999999860.9985251963218283 
State_D8 0.99851388008865440.99999999999986120.99853832113312 
State_D9 0.99789548674089160.99999999999998580.9979003500387397 
State_D10 0.99899482220359910.99999999999974090.9990171904745906 
State_D11 0.99882898529036370.99999999999978530.9988546978789684 
State_D12 0.99883370462813540.99999999999978620.9988596121105079 
State_D13 0.99809719710336380.99999999999993630.9981112105634102 
State_D14 0.9981294057959960.99999999999993690.9981440321284472 
State_D15 0.9987200680942190.99999999999981450.998746347781682 
State_D16 0.99872051243386570.9999999999998150.9987466551479767 
State_D17 0.99850183419307970.99999999999985930.9985259745179441 
State_D18 0.99851380960311

State_D1 0.91124510601931090.99999999999999730.9520968621684825 
State_D2 0.9119663713130960.99999999999999820.9714493914139383 
State_D3 0.91487693128799170.99999999999999740.9643825431065961 
State_D4 0.93620316440051480.99999999999999820.9911770399640951 
State_D5 0.91252155500886210.99999999999999770.9757182101948781 
State_D6 0.91252155500415010.99999999999999790.9295938129872479 
State_D7 0.91883893874244750.99999999999999730.9396039531429446 
State_D8 0.91313353040438890.99999999999999780.9476317161816921 
State_D9 0.91105351726581320.99999999999999780.9744048697460826 
State_D10 0.91609992316778430.99999999999999810.9907449251893821 
State_D11 0.91487693128799170.99999999999999770.9211249809398783 
State_D12 0.93620316440096230.99999999999999840.9822343986738409 
State_D13 0.91124993894953730.99999999999999730.8984767018240702 
State_D14 0.9119663713146580.99999999999999830.9777834162456233 
State_D15 0.93089575256946620.99999999999999810.9532970861941954 
State_D16 0.930895752

In [11]:
tests_data_file_name

"POP_NoRmT0d0LME_trn4_gap_tst20_T0_diag0_fix-Phi-up_2023-Aug-10_at_17-26.h5"